In [ ]:
# | default_exp types

In [ ]:
# | export
import re
from dataclasses import dataclass
from typing import Optional

In [ ]:
# | hide
import nbdev

In [ ]:
# | export
@dataclass
class Ticker:
    symbol: str
    exchange: str

In [ ]:
# | export
@dataclass
class FilingMetadata:
    accession_number: str
    form_type: str
    primary_doc_url: str
    items: str
    primary_doc_description: str
    filing_date: str
    report_date: str
    cik: str
    company_name: str
    tickers: list[Ticker]

In [ ]:
# | export
@dataclass
class RequestedFilings:
    ticker_or_cik: str
    form_type: str = "10-Q"
    limit: int = 1

    _REGEX_PATTERN = r"^(?:(\d+)/)?([^/]+)(?:/(.+))?$"

    @classmethod
    def from_string(cls, s: str):
        match = re.match(cls._REGEX_PATTERN, s.strip())
        if not match:
            raise ValueError(f"Invalid RequestedFilings string: {s}")
        limit_str, ticker_or_cik, form_type = match.groups()
        limit = int(limit_str) if limit_str else cls.limit
        form_type = form_type if form_type else cls.form_type
        form_type = form_type.upper()
        return cls(
            limit=limit,
            ticker_or_cik=ticker_or_cik,
            form_type=form_type,
        )

In [ ]:
# Define test cases and expected results
test_cases = [
    ("3/AAPL/10-Q", RequestedFilings(limit=3, ticker_or_cik="AAPL", form_type="10-Q")),
    ("MSFT/10-K", RequestedFilings(ticker_or_cik="MSFT", form_type="10-K")),
    ("1/GOOG", RequestedFilings(limit=1, ticker_or_cik="GOOG")),
    ("BRK.A", RequestedFilings(ticker_or_cik="BRK.A")),
    ("1/0001990966/SC TO-T", RequestedFilings(limit=1, ticker_or_cik="0001990966", form_type="SC TO-T")),
    ("1/0001990966/SC TO-T/A", RequestedFilings(limit=1, ticker_or_cik="0001990966", form_type="SC TO-T/A")),
]

# Run test cases and assert expected results
for test_string, expected in test_cases:
    result = RequestedFilings.from_string(test_string)
    assert (
        result == expected
    ), f"Test case failed for input: {test_string}. Expected: {expected}, Got: {result}"

In [ ]:
# | export
@dataclass
class CompanyAndAccessionNumber:
    ticker_or_cik: str
    accession_number: str

    _REGEX_PATTERN = re.compile(r"\b([A-Za-z0-9.]+)/(\d{10}-?\d{2}-?\d{6})\b")

    @classmethod
    def from_string(
        cls, s: str, *, must_match=False
    ) -> Optional["CompanyAndAccessionNumber"]:
        match = re.search(cls._REGEX_PATTERN, s.strip())
        if not match:
            if must_match:
                raise ValueError(f"Invalid RequestedFilings string: {s}")
            else:
                return None
        ticker_or_cik, accession_number = match.groups()
        return cls(
            ticker_or_cik=ticker_or_cik,
            accession_number=accession_number,
        )

In [ ]:
test_cases = [
    (
        "AAPL/0000320193-20-000052",
        CompanyAndAccessionNumber(
            ticker_or_cik="AAPL", accession_number="0000320193-20-000052"
        ),
    ),
    (
        "0001193125/000119312520036918",
        CompanyAndAccessionNumber(
            ticker_or_cik="0001193125", accession_number="000119312520036918"
        ),
    ),
    (
        "1193125/000119312520036918",
        CompanyAndAccessionNumber(
            ticker_or_cik="1193125", accession_number="000119312520036918"
        ),
    ),
    ("INVALID_STRING", None),
]

for test_input, expected in test_cases:
    result = CompanyAndAccessionNumber.from_string(test_input, must_match=False)
    assert (
        result == expected
    ), f"Test case failed for input: {test_input}. Expected: {expected}, Got: {result}"

In [ ]:
# | hide
nbdev.nbdev_export()